In [ ]:
import pandas as pd
import os

In [ ]:
pwd = os.getcwd() # Transforma o diretorio atual em uma string | PWD = Present Working Directory

In [ ]:
dataset = pd.read_excel(pwd + "\\Data - Survey Monkey Output Edited.xlsx", sheet_name="Edited_Data")
dataset

In [ ]:
dataset_modified = dataset.copy()
dataset_modified

In [ ]:
columns_to_drop = ['Start Date - ', 'End Date - ', 'Email Address - ',
       'First Name - ', 'Last Name - ', 'Custom Data 1 - ']
columns_to_drop

In [ ]:
dataset_modified = dataset_modified.drop(columns=columns_to_drop)
dataset_modified

In [ ]:
id_vars = list(dataset_modified.columns)[ :8 ]
value_vars = list(dataset_modified.columns)[8 : ]
# value_vars

In [ ]:
dataset_melted = dataset_modified.melt(id_vars = id_vars, value_vars=value_vars, var_name="Question + Subquestion",value_name="Answer")
dataset_melted

In [ ]:
questions_import = pd.read_excel(pwd + "\\Data - Survey Monkey Output Edited.xlsx", sheet_name="Question")
questions_import

In [ ]:
questions = questions_import.copy()
questions.drop(columns=["Raw Question",	"Raw Subquestion",	"Subquestion"],inplace=True)
questions

In [ ]:
questions.dropna(inplace=True)


In [ ]:
questions

In [ ]:
dataset_merged = pd.merge(left=dataset_melted, right=questions, how="left",left_on="Question + Subquestion",right_on="Question + Subquestion")
print("Original Data", len(dataset_melted))
print("Merged Data", len(dataset_merged))
dataset_merged

In [ ]:
respondents = dataset_merged[dataset_merged["Answer"].notna()]
respondents = respondents.groupby("Question")["Respondent ID - "].nunique().reset_index()
respondents.rename(columns={"Respondent ID - ":"Respondent"}, inplace=True)
respondents

In [ ]:
dataset_merged_two = pd.merge(left=dataset_merged, right=respondents, how="left",left_on="Question",right_on="Question")
print("Original Data", len(dataset_melted))
print("Merged Data", len(dataset_merged_two))
dataset_merged_two

In [ ]:
same_answer = dataset_merged # [dataset_merged["Answer"].notna()]
same_answer = same_answer.groupby(["Question + Subquestion","Answer"])["Respondent ID - "].nunique().reset_index()
same_answer.rename(columns={"Respondent ID - ":"Same Answer"}, inplace=True)
same_answer

In [88]:
dataset_merged_three = pd.merge(left=dataset_merged_two, right=same_answer, how="left",left_on=["Question + Subquestion","Answer"],right_on=["Question + Subquestion","Answer"])
dataset_merged_three["Same Answer"].fillna(0, inplace=True)
print("Original Data", len(dataset_merged_two))
print("Merged Data", len(dataset_merged_three))
dataset_merged_three

Original Data 17028
Merged Data 17028


,Respondent ID -,Identify which division you work in. - Response,Identify which division you work in. - Other (please specify),Which of the following best describes your position level? - Response,Which generation are you apart of? - Response,Please select the gender in which you identify. - Response,Which duration range best aligns with your tenure at your company? - Response,Which of the following best describes your employment type? - Response,Question + Subquestion,Answer,Question,Respondent,Same Answer
0,5379192392,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1 - Response,NaN,Question 1,119,0.0
1,2658722536,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Question 1 - Response,Answer 4,Question 1,119,17.0
2,4044163394,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,Answer 5,Question 1,119,22.0
3,5535865599,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1 - Response,Answer 1,Question 1,119,14.0
4,3356802928,Port Operations,NaN,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1 - Response,NaN,Question 1,119,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17023,7940065082,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30 - Response 3,Answer 8,Question 30,182,14.0
17024,5157705612,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30 - Response 3,Answer 6,Question 30,182,20.0
17025,9920755555,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN,Question 30,182,0.0
17026,6638341389,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN,Question 30,182,0.0


In [89]:
output = dataset_merged_three.copy()
output.rename(columns={"Identify which division you work in. - Response":"Primary Division" , "Identify which division you work in. - Other (please specify)":"Secondary Division" , "Which of the following best describes your position level? - Response":"Position" , "Which generation are you apart of? - Response":"Generation" , "Please select the gender in which you identify. - Response":"Gender" , "Which duration range best aligns with your tenure at your company? - Response":"Tenure" , "Which of the following best describes your employment type? - Response":"Employment Type"},inplace=True)
output

,Respondent ID -,Primary Division,Secondary Division,Position,Generation,Gender,Tenure,Employment Type,Question + Subquestion,Answer,Question,Respondent,Same Answer
0,5379192392,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1 - Response,NaN,Question 1,119,0.0
1,2658722536,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Question 1 - Response,Answer 4,Question 1,119,17.0
2,4044163394,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1 - Response,Answer 5,Question 1,119,22.0
3,5535865599,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1 - Response,Answer 1,Question 1,119,14.0
4,3356802928,Port Operations,NaN,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1 - Response,NaN,Question 1,119,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17023,7940065082,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30 - Response 3,Answer 8,Question 30,182,14.0
17024,5157705612,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30 - Response 3,Answer 6,Question 30,182,20.0
17025,9920755555,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN,Question 30,182,0.0
17026,6638341389,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30 - Response 3,NaN,Question 30,182,0.0


In [91]:
output.to_excel(pwd + "\Final_Output.xlsx", index=False)